## Backtracking (TLE)
- Backtrack until last row and last col are reached
- At every recursive step, set the current height to 0 to mark it as in the current path
- Take the minimum difference for each neighbor and return it

In [1]:
import math

class Solution:
    def minimum_effort_path(self, heights):
        rows, cols = len(heights), len(heights[0])

        def dfs(row, col, max_difference):
            if row == rows - 1 and col == cols - 1:
                return max_difference

            height = heights[row][col]
            heights[row][col] = 0
            min_difference = math.inf

            for dx, dy in ((0, -1), (0, 1), (-1, 0), (1, 0)):
                new_row, new_col = row + dy, col + dx

                if 0 <= new_row < rows and 0 <= new_col < cols and heights[new_row][new_col] != 0:
                    cur_difference = abs(heights[new_row][new_col] - height)
                    max_cur_difference = max(max_difference, cur_difference)
                    result = dfs(new_row, new_col, max_cur_difference)
                    min_difference = min(min_difference, result)
            
            heights[row][col] = height
            return min_difference

        return dfs(0, 0, 0)

## Djikstra's Algorithm
- Essentially finding shortest path from source to destination
- Djikstra's used to find shortest path in weighted graph, so can be used here

In [3]:
from typing import List
import heapq

class Solution:
    def minimumEffortPath(self, heights: List[List[int]]) -> int:
        rows, cols = len(heights), len(heights[0])
        
        differences = [[math.inf] * cols for _ in range(rows)]
        differences[0][0] = 0
        visited = set([(0, 0)])

        heap = [(0, 0, 0)]
        while heap:
            difference, row, col = heapq.heappop(heap)
            visited.add((row, col))

            for dx, dy in ((0, -1), (0, 1), (-1, 0), (1, 0)):
                new_row, new_col = row + dy, col + dx
                if 0 <= new_row < rows and 0 <= new_col < cols and (new_row, new_col) not in visited:
                    cur_difference = max(difference, abs(heights[new_row][new_col] - heights[row][col]))
                    if cur_difference < differences[new_row][new_col]:
                        differences[new_row][new_col] = cur_difference
                        heapq.heappush(heap, (cur_difference, new_row, new_col))
        
        return differences[-1][-1]

## Union Find
- Initialize union find of size `row * col` where each cell `(cur_row, cur_col)` is stored at `(cur_row * col + cur_col)`
- Heap to keep track of absolute difference between every adjacent cell
- Iterate over sorted edge list and connect each edge to form connected component
- After each union, check if source and destination are connected
- If yes, absolute difference is answer

In [2]:
from typing import List

class UnionFind:
    def __init__(self, size):
        self.roots = [i for i in range(size)]
        self.ranks = [0] * size
    
    def find(self, node):
        if self.roots[node] != node:
            self.roots[node] = self.find(self.roots[node])

        return self.roots[node]
    
    def union(self, node1, node2):
        root1 = self.find(node1)
        root2 = self.find(node2)

        if root1 != root2:
            if self.ranks[root1] < self.ranks[root2]:
                self.roots[root1] = root2
            elif self.ranks[root1] > self.ranks[root2]:
                self.roots[root2] = root1
            else:
                self.roots[root2] = root1
                self.ranks[root1] += 1
    
    def are_connected(self, node1, node2):
        return self.find(node1) == self.find(node2)

class Solution:
    def minimumEffortPath(self, heights: List[List[int]]) -> int:
        rows, cols = len(heights), len(heights[0])

        if rows == cols == 1:
            return 0

        edges = []
        for cur_row in range(rows):
            for cur_col in range(cols):
                if cur_row > 0:
                    node1 = cur_row * cols + cur_col
                    node2 = (cur_row - 1) * cols + cur_col
                    diff = abs(heights[cur_row][cur_col] - heights[cur_row - 1][cur_col])
                    edges.append((diff, node1, node2))
                if cur_col > 0:
                    node1 = cur_row * cols + cur_col
                    node2 = cur_row * cols + cur_col - 1
                    diff = abs(heights[cur_row][cur_col] - heights[cur_row][cur_col - 1])
                    edges.append((diff, node1, node2))
        edges.sort()

        union_find = UnionFind(rows * cols)

        for diff, node1, node2 in edges:
            union_find.union(node1, node2)
            if union_find.are_connected(0, rows * cols - 1):
                return diff
        
        return -1

## BFS Binary Search
- Max height is $10^6$, so can use binary search
- Use BFS to determine if there exists path from source to destination for `mid` value

In [3]:
class Solution:
    def minimumEffortPath(self, heights: List[List[int]]) -> int:
        rows, cols = len(heights), len(heights[0])

        def viable(mid):
            edges = [(0, 0)]
            visited = set([(0, 0)])

            while edges:
                row, col = edges.pop()
                if row == rows - 1 and col == cols - 1:
                    return True
                
                for dx, dy in ((0, -1), (0, 1), (-1, 0), (1, 0)):
                    new_row, new_col = row + dy, col + dx
                    if 0 <= new_row < rows and 0 <= new_col < cols and (new_row, new_col) not in visited:
                        cur_diff = abs(heights[row][col] - heights[new_row][new_col])
                        if cur_diff <= mid:
                            edges.append((new_row, new_col))
                            visited.add((new_row, new_col))
            
            return False

        left, right = 0, 10 ** 6
        while left < right:
            mid = (left + right) // 2
            if viable(mid):
                right = mid
            else:
                left = mid + 1
        return left

## DFS Binary Search

In [4]:
class Solution:
    def minimumEffortPath(self, heights: List[List[int]]) -> int:
        rows, cols = len(heights), len(heights[0])

        def viable(row, col, mid):
            if row == rows - 1 and col == cols - 1:
                return True
            
            for dx, dy in ((0, -1), (0, 1), (-1, 0), (1, 0)):
                new_row, new_col = row + dy, col + dx

                if 0 <= new_row < rows and 0 <= new_col < cols and (new_row, new_col) not in visited:
                    diff = abs(heights[row][col] - heights[new_row][new_col])
                    if diff <= mid:
                        visited.add((new_row, new_col))
                        if viable(new_row, new_col, mid):
                            return True
            
            return False

        left, right = 0, 10 ** 6
        while left < right:
            mid = (left + right) // 2
            visited = set([(0, 0)])
            if viable(0, 0, mid):
                right = mid
            else:
                left = mid + 1
        return left